In [51]:
import numpy as np
import pandas as pd
from faker import Faker
import random
import matplotlib.pyplot as plt
import secrets
import string
import psycopg2

# Fake client data

In [32]:
def getRandomPasswordString():
    alphabet = string.ascii_letters + string.digits + string.punctuation.replace(',', '')  # Exclure la virgule

    password = secrets.choice(string.ascii_uppercase) + \
               secrets.choice(string.ascii_lowercase) + \
               secrets.choice(string.digits) + \
               secrets.choice(string.punctuation.replace(',', '')) + \
               ''.join(secrets.choice(alphabet) for i in range(12))

    return password


In [33]:
getRandomPasswordString()

'Vn2"$5jJW>G:[aPM'

In [34]:
def generate_data_1():
    fake = Faker()
    liste_des_domaines = ['com','net','org','gov','fr']

    list_rows = []
    nb_row = random.randint(10000, 12000)

    generated_ids = set()   # Utilisation d'un ensemble pour stocker les IDs générés

    for i in range(nb_row):
        first_name = fake.first_name()
        last_name = fake.last_name()
        company = fake.company().split()[0].strip(',')
        dns_org = fake.random_choices(elements=liste_des_domaines, length=1)[0]
        email = f"{first_name}.{last_name}@{company}.{dns_org}".lower()
        unique_id = random.randint(100000, 999999)
        age = random.randint(18, 80)
        pwd = getRandomPasswordString()

        if unique_id not in generated_ids:
            generated_ids.add(unique_id)  # Ajoute l'ID à l'ensemble des IDs générés
            list_rows.append(
                (unique_id, first_name, last_name, pwd, age, fake.job(), fake.address(), fake.phone_number(), email)
            )
    return list_rows

In [35]:
data = generate_data_1()

In [57]:
client = pd.DataFrame(data, columns=['id', 'prenom', 'nom', 'pwd', 'age', 'job', 'adress', 'phoneNumber', 'email'])
client.head()

,id,prenom,nom,pwd,age,job,adress,phoneNumber,email
0,953125,Christopher,Cantu,Yn5*@%S_&BvwnZ:o,74,Broadcast journalist,USCGC Jones\nFPO AE 91044,001-488-606-0601x459,christopher.cantu@anderson.net
1,891182,Sandra,Oconnor,Sf4}XM.*;7qhX[Dx,46,Chief Technology Officer,"7312 Kathy Trace\nNorth Benjamin, UT 41664",+1-900-681-6671x82178,sandra.oconnor@griffin.org
2,534904,Bradley,Fisher,Ac7{E&!;.AR!1F~C,27,Risk analyst,"80960 Alvarado Harbor\nEast Joshuachester, AS ...",001-919-985-1035,bradley.fisher@jimenez.fr
3,497007,Carla,Wright,Zy1<9gyY&V<@<O%t,67,Adult guidance worker,"20946 Crystal Forge Apt. 175\nEast Jessica, NE...",(602)528-1210x649,carla.wright@mathis-jones.org
4,839432,Kelsey,Gross,Aq4&gX'r*);vab9A,68,Politician's assistant,"51428 Parker Falls Suite 370\nEast Rickyville,...",+1-401-348-1059x65185,kelsey.gross@schneider.com


In [37]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11436 entries, 0 to 11435
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           11436 non-null  int64 
 1   prenom       11436 non-null  object
 2   nom          11436 non-null  object
 3   pwd          11436 non-null  object
 4   age          11436 non-null  int64 
 5   job          11436 non-null  object
 6   adress       11436 non-null  object
 7   phoneNumber  11436 non-null  object
 8   email        11436 non-null  object
dtypes: int64(2), object(7)
memory usage: 804.2+ KB


In [38]:
client.to_csv('client.csv', index=False)

# Fake game stats data

In [66]:
# Connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    dbname="jeux-videos",
    user="postgres",
    password="S0314",
    host="localhost",
    port="5432"
)

# Définition de la requête SQL pour récupérer toutes les colonnes de la table 'article'
query = "SELECT id FROM article"

# Lecture des données de la table 'article' dans une DataFrame Pandas
dataframe = pd.read_sql(query, conn)

# Fermeture de la connexion
conn.close()


C:\Users\PC de MEDUSA\AppData\Local\Temp\ipykernel_28240\2890172480.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, conn)


In [79]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      38 non-null     int64
dtypes: int64(1)
memory usage: 436.0 bytes


In [76]:
# Affichage des premières lignes de la DataFrame
l = list(dataframe['id']) 
ll = list(np.arange(2, 52))

# Convertir les listes en ensembles
elements_absents = [element for element in ll if element not in l]

In [85]:
# Générer les IDs de 2 à 50
all_ids = np.arange(2, 52)
excluded_ids = [8, 11, 12, 13, 14, 15, 16, 27, 28, 29, 34, 45]
id_values = [e for e in all_ids if e not in excluded_ids]

np.random.seed(42)
qte_ajoutee_values = np.random.randint(2000, 8001, size=38)  # Générer des valeurs aléatoires entre 2000 et 9000
qte_achetee_values = np.random.randint(1000, qte_ajoutee_values, size=38)  # Générer des valeurs aléatoires supéreiru à 1000 et inférieures à qte_ajoutee

# Créer la DataFrame avec les colonnes spécifiées
table = {
    'id': id_values,
    'qte_ajoutee': qte_ajoutee_values,
    'qte_achetee': qte_achetee_values
}
jeux_stats = pd.DataFrame(table)


In [87]:
jeux_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   id           38 non-null     int32
 1   qte_ajoutee  38 non-null     int32
 2   qte_achetee  38 non-null     int32
dtypes: int32(3)
memory usage: 588.0 bytes


In [88]:
jeux_stats.to_csv('jeux_stats.csv', index=False)